# Image file collection 
python version -31.11.5

## Run for authen

In [42]:
import ee 
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-qinheyi')

## Load map

In [43]:
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

m=geemap.Map()


def getDateRange(imageCollection:ee.ImageCollection):
    dateRange=imageCollection.aggregate_array('system:time_start')
    date_list=[ee.Date(date).format().getInfo() for date in dateRange.getInfo()]
    print(date_list)
    return date_list

# imageRange=getDateRange(nicfi) # because of the server-side function.

baseMap=nicfi.filter(ee.Filter.date('2024-01-01','2025-01-11')).sort('system:time_start', False).first()

m.centerObject(baseMap,4)

vis={'bands':['R','G','B'],'min':64,'max':5454,'gamma':1.8}
m.addLayer(baseMap,vis,'nicfi')
m.height='800px'

display(m)


Map(center=[-0.0013335310118573354, -73.54611700899127], controls=(WidgetControl(options=['position', 'transpa…

## Add shape file and show one


In [44]:
# get the table directly from my projcet
shapefile_table=ee.FeatureCollection('projects/ee-qinheyi/assets/1823_ADRSM')
print('Count of table item:',shapefile_table.size().getInfo())
# getInfo function is a request from the google servers it's server-side function and getback to the result.

shape_style={
  'color': 'FF0000',  # Outline color (in hex format, here it's red)
  'width': 2,          # Outline width
  'fillColor':'00000000'
}

field_name='Index'
desired_value=1345 #346
filtered_table=shapefile_table.filter(ee.Filter.eq(field_name,desired_value))
selected_geometry=filtered_table.geometry()


selected_fc = filtered_table.style( # this is a image type
    color='FF0000',width=2,fillColor='00000000',
)

m.addLayer(selected_fc,{},'Selected Geometry')
m.centerObject(selected_geometry)
display(m)


Count of table item: 1823


Map(bottom=2448.0, center=[-0.0013335310118573354, -73.54611700899127], controls=(WidgetControl(options=['posi…

## get the wdiget Screenshot by `ipywebrtc`

In [49]:
from ipywebrtc import WidgetStream,ImageRecorder

map_stream = WidgetStream(widget=m, max_fps=1) # the widgetStream can only on the local machine. if on the colab it can be no view rendered.


recorder = ImageRecorder(stream=map_stream)
recorder
recorder.recording=True
recorder.autosave = False
recorder.image # something erro black image     





Image(value=b'')

## Conclusion:

Through colab, we could not dircetly get the jpgs through either of the Geemap and the ipywebrtc. 

We decide using arcpy and arcgis pro to export the combination of the images